This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***



 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [1]:
DATA_PATH = './obesity_data/'

**Classical Machine Learning - TF-IDF - All Features**

![CML TFIDF All](images\cml-tfidf-all.gif)

**Classical Machine Learning - TF-IDF - ExtraTreesClassifier Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-extra.gif)

**Classical Machine Learning - TF-IDF - InfoGain Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-infogain.gif)

**Classical Machine Learning - TF-IDF - SelectKBest Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-selectkbest.gif)

**Classical Machine Learning - Word Embeddings - No Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swno.gif)

**Classical Machine Learning - Word Embeddings - Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swyes.gif)

In [2]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime
from datetime import timedelta
from tqdm import tqdm
import torchtext
from nltk.corpus import stopwords
import tensorflow_hub as hub

from sklearn.metrics import roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, svm, naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = './obesity_data/'
RESULTS_PATH = './results/'
MODELS_PATH = './models/'

cachedStopWords = stopwords.words("english")

#Download info for USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

all_docs_df = pd.read_pickle(DATA_PATH + '/alldocs_df.pkl')
all_docs_df_ns = pd.read_pickle(DATA_PATH + '/alldocs_df_ns.pkl')
all_annot_df = pd.read_pickle(DATA_PATH + '/allannot_df.pkl')
all_df_expanded = pd.read_pickle(DATA_PATH + '/all_df_expanded.pkl')

C:\Users\mclop\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
all_df = pd.merge(all_docs_df,all_annot_df, on='id')
all_df_ns = pd.merge(all_docs_df_ns,all_annot_df, on='id')

disease_list = all_df['disease'].unique().tolist()
feature_list = ['All','ExtraTreeClassifier','SelectKBest','InfoGainAttributeVal']
embedding_list = ['GloVe', 'FastText', 'USE']
device = torch.device('cpu')

In [4]:
for index, entry in enumerate(all_df['tok_lem_text']):
    Final_words = []
    for word in entry:
        Final_words.append(word)
    all_df.loc[index, 'text_final'] = str(Final_words)

In [5]:
for index, entry in enumerate(all_df_ns['tok_lem_text']):
    Final_words = []
    for word in entry:
        Final_words.append(word)
    all_df_ns.loc[index, 'text_final'] = str(Final_words)

In [6]:
def performSVM(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf, y_train)

    predictions_SVM = SVM.predict(Test_X_Tfidf)

    #f1 = f1_score(y_test, predictions_SVM)
    f1_macro = f1_score(y_test, predictions_SVM,average='macro')
    f1_micro = f1_score(y_test, predictions_SVM,average='micro')

    #print("SVM - ", disease, ": f1-score", f1)
    print("SVM - ", feature, disease, ": f1-macro", f1_macro)
    print("SVM - ", feature, disease, ": f1-micro", f1_micro)

In [7]:
def performKNN(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    knn = KNeighborsClassifier(n_neighbors=7)
    clf = knn.fit(Train_X_Tfidf, y_train)

    # predict the labels on validation dataset
    predictions_KNN = clf.predict(Test_X_Tfidf)

    #auroc = roc_auc_score(truth, pred[:,1])
    #f1 = f1_score(y_test, predictions_KNN)
    f1_macro = f1_score(y_test, predictions_KNN,average='macro')
    f1_micro = f1_score(y_test, predictions_KNN,average='micro')

    #print("KNN - ", disease, ": f1-score", f1)
    print("KNN - ", feature, disease, ": f1-macro", f1_macro)
    print("KNN - ", feature, disease, ": f1-micro", f1_micro)

In [8]:
def performNB(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf,y_train)

    # predict the labels on validation dataset
    predictions_NB = Naive.predict(Test_X_Tfidf)

    #f1 = f1_score(y_test, predictions_NB)
    f1_macro = f1_score(y_test, predictions_NB,average='macro')
    f1_micro = f1_score(y_test, predictions_NB,average='micro')

    #print("NB - ", disease, ": f1-score", f1)
    print("NB - ", feature, disease, ": f1-macro", f1_macro)
    print("NB - ", feature, disease, ": f1-micro", f1_micro)

In [9]:
def performRF(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    classifier=RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =0)
    classifier.fit(Train_X_Tfidf,y_train)

    # predict the labels on validation dataset
    predictions_RF = classifier.predict(Test_X_Tfidf)

    #f1 = f1_score(y_test, predictions_RF)
    f1_macro = f1_score(y_test, predictions_RF,average='macro')
    f1_micro = f1_score(y_test, predictions_RF,average='micro')

    #print("RF - ", disease, ": f1-score", f1)
    print("RF - ", feature, disease, ": f1-macro", f1_macro)
    print("RF - ", feature, disease, ": f1-micro", f1_micro)

In [10]:
def vectorize_batch_GloVe(X_Train):
    embedding_size_used = 300
    vec = torchtext.vocab.GloVe(name='6B', dim=embedding_size_used)    
    
    X = [0] * X_Train.shape[0]
    
    for i in range(len(X_Train)):
        vectors = vec.get_vecs_by_tokens(X_Train.iloc[i]).float().numpy()

        X[i] = vectors.flatten()

    return X

In [11]:
def vectorize_batch_FastText(X_Train):
    embedding_size_used = 300
    vec = torchtext.vocab.FastText()

    X = [0] * X_Train.shape[0]

    for i in range(len(X_Train)):
        vectors = vec.get_vecs_by_tokens(X_Train.iloc[i]).float().numpy()

        X[i] = vectors.flatten()

    return X

In [12]:
def vectorize_batch_USE(X_Train):
    embedding_size_used = 512

    X = [0] * X_Train.shape[0]

    for i in range(len(X_Train)):
        
        tensor_flow_vectors = embed(x)
        array_vectors = tensor_flow_vectors.numpy()

        X[i] = torch.tensor(array_vectors).float()
        Y[i] = torch.tensor(float(y == True))

    return X,Y 

In [13]:
def generateXTrainAndTest(X_train, X_test, Tfidf_vect):
    X_train_values_list = Tfidf_vect.fit_transform(X_train).toarray()
    X_training = pd.DataFrame(X_train_values_list, columns=Tfidf_vect.get_feature_names_out())
    X_training = np.asarray(X_training, dtype=float)
    X_training = torch.from_numpy(X_training).to(device)

    X_test_values_list = Tfidf_vect.transform(X_test).toarray()
    X_testing = pd.DataFrame(X_test_values_list, columns=Tfidf_vect.get_feature_names_out())
    X_testing = np.asarray(X_testing, dtype=float)
    X_testing = torch.from_numpy(X_testing).to(device)  
    
    return X_training, X_testing

In [14]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV, RFE
from sklearn.tree import ExtraTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import f_classif, mutual_info_classif

def getVocab(X_train, y_train, feature, max_tokens):
 
    ## Step 1: Determine the Initial Vocabulary
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    vocab = list(tokenizer.word_index.keys())

    ## Step 2: Create term  matrix
    vectors = tokenizer.texts_to_matrix(X_train, mode='count')

    ## Do feature selection on term matrix (column headers are words)
    X = vectors
    y = y_train

    ##Choose algorithm
    if feature == 'SelectKBest':
        selector = SelectKBest(score_func=f_classif, k=max_tokens).fit(X,y)
    else: 
        if feature == 'InfoGainAttributeVal':
            #This should be similar to the InfoGain?
            selector = SelectKBest(score_func=mutual_info_classif, k=max_tokens).fit(X,y)
        else:
            #default to ExtraTreeClassifier
            estimator = ExtraTreeClassifier(random_state = seed)
            selector = SelectFromModel(estimator, max_features = max_tokens)
            selector = selector.fit(X, y)

    support_idx = selector.get_support(True)
    tokenizer2 = Tokenizer()
    tokenizer2.fit_on_texts([vocab[i-1].replace("'","") for i in support_idx])
    new_vocab = list(tokenizer2.word_index.keys())

    return new_vocab

# Feature Selection

In [15]:
Encoder = LabelEncoder()
Tfidf_vect = TfidfVectorizer(max_features=600)
Tfidf_vect_NS = TfidfVectorizer(max_features = 600, stop_words = cachedStopWords)

max_tokens = 600

for _,disease in enumerate(disease_list):
    for _,feature in enumerate(feature_list):
        disease_data_df = all_df[all_df['disease'] == disease]

        X_train, X_test, y_train, y_test = train_test_split(disease_data_df['text_final'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
        
        if feature != 'All':
            vocab = getVocab(X_train,y_train, feature, max_tokens)
            Tfidf_vect = TfidfVectorizer(max_features=max_tokens,vocabulary = vocab)
        else:
            Tfidf_vect = TfidfVectorizer(max_features=max_tokens)
  
        
        X_train_values_list = Tfidf_vect.fit_transform(X_train).toarray()
        X_training = pd.DataFrame(X_train_values_list, columns=Tfidf_vect.get_feature_names_out())
        X_training = np.asarray(X_training, dtype=float)
        X_training = torch.from_numpy(X_training).to(device)

        X_test_values_list = Tfidf_vect.transform(X_test).toarray()
        X_testing = pd.DataFrame(X_test_values_list, columns=Tfidf_vect.get_feature_names_out())
        X_testing = np.asarray(X_testing, dtype=float)
        X_testing = torch.from_numpy(X_testing).to(device)
        
        tokens_to_use = X_training.shape[1]

        Train_Y  = Encoder.fit_transform(y_train)
        Test_Y  = Encoder.fit_transform(y_test)

        performSVM(X_training, X_testing, Train_Y, Test_Y, feature)

        performKNN(X_training, X_testing, Train_Y, Test_Y, feature)

        performNB(X_training, X_testing, Train_Y, Test_Y, feature)

        performRF(X_training, X_testing, Train_Y, Test_Y, feature)
        

SVM -  All Asthma : f1-macro 0.609237775127399
SVM -  All Asthma : f1-micro 0.9198113207547169
KNN -  All Asthma : f1-macro 0.5846627483907081
KNN -  All Asthma : f1-micro 0.9009433962264151
NB -  All Asthma : f1-macro 0.6466666666666667
NB -  All Asthma : f1-micro 0.9245283018867925
RF -  All Asthma : f1-macro 0.6466666666666667
RF -  All Asthma : f1-micro 0.9245283018867925
SVM -  ExtraTreeClassifier Asthma : f1-macro 0.6002514142049027
SVM -  ExtraTreeClassifier Asthma : f1-micro 0.8726415094339622
KNN -  ExtraTreeClassifier Asthma : f1-macro 0.49066108656901497
KNN -  ExtraTreeClassifier Asthma : f1-micro 0.8537735849056604
NB -  ExtraTreeClassifier Asthma : f1-macro 0.4953623902158746
NB -  ExtraTreeClassifier Asthma : f1-micro 0.8632075471698113
RF -  ExtraTreeClassifier Asthma : f1-macro 0.5750286368843069
RF -  ExtraTreeClassifier Asthma : f1-micro 0.8679245283018869


C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


SVM -  SelectKBest Asthma : f1-macro 0.8923311325545962
SVM -  SelectKBest Asthma : f1-micro 0.9433962264150944
KNN -  SelectKBest Asthma : f1-macro 0.6476063829787234
KNN -  SelectKBest Asthma : f1-micro 0.8584905660377359
NB -  SelectKBest Asthma : f1-macro 0.5004776157422148
NB -  SelectKBest Asthma : f1-micro 0.8254716981132075
RF -  SelectKBest Asthma : f1-macro 0.8110853680270896
RF -  SelectKBest Asthma : f1-micro 0.9056603773584906
SVM -  InfoGainAttributeVal Asthma : f1-macro 0.8885223959080262
SVM -  InfoGainAttributeVal Asthma : f1-micro 0.9481132075471698
KNN -  InfoGainAttributeVal Asthma : f1-macro 0.6892406918792143
KNN -  InfoGainAttributeVal Asthma : f1-micro 0.8820754716981133
NB -  InfoGainAttributeVal Asthma : f1-macro 0.5706404078497103
NB -  InfoGainAttributeVal Asthma : f1-micro 0.8632075471698113
RF -  InfoGainAttributeVal Asthma : f1-macro 0.8479850853291266
RF -  InfoGainAttributeVal Asthma : f1-micro 0.9292452830188679
SVM -  All CHF : f1-macro 0.844006568144

C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


SVM -  SelectKBest CHF : f1-macro 0.8228181876648016
SVM -  SelectKBest CHF : f1-micro 0.8689655172413793
KNN -  SelectKBest CHF : f1-macro 0.6610389610389611
KNN -  SelectKBest CHF : f1-micro 0.7517241379310344
NB -  SelectKBest CHF : f1-macro 0.5607626514611547
NB -  SelectKBest CHF : f1-micro 0.7655172413793103
RF -  SelectKBest CHF : f1-macro 0.872997997997998
RF -  SelectKBest CHF : f1-micro 0.903448275862069
SVM -  InfoGainAttributeVal CHF : f1-macro 0.9243846474760118
SVM -  InfoGainAttributeVal CHF : f1-micro 0.9310344827586207
KNN -  InfoGainAttributeVal CHF : f1-macro 0.6754662840746055
KNN -  InfoGainAttributeVal CHF : f1-micro 0.7310344827586208
NB -  InfoGainAttributeVal CHF : f1-macro 0.3958333333333333
NB -  InfoGainAttributeVal CHF : f1-micro 0.6551724137931034
RF -  InfoGainAttributeVal CHF : f1-macro 0.9065721649484535
RF -  InfoGainAttributeVal CHF : f1-micro 0.9172413793103448
SVM -  All Depression : f1-macro 0.42933333333333334
SVM -  All Depression : f1-micro 0.75

...

SVM -  SelectKBest PVD : f1-macro 0.6494611457742484
SVM -  SelectKBest PVD : f1-micro 0.8689320388349513
KNN -  SelectKBest PVD : f1-macro 0.536166619757951
KNN -  SelectKBest PVD : f1-micro 0.8446601941747571
NB -  SelectKBest PVD : f1-macro 0.4535809018567639
NB -  SelectKBest PVD : f1-micro 0.8300970873786409
RF -  SelectKBest PVD : f1-macro 0.7406135531135531
RF -  SelectKBest PVD : f1-micro 0.8932038834951457
SVM -  InfoGainAttributeVal PVD : f1-macro 0.8531717747683535
SVM -  InfoGainAttributeVal PVD : f1-micro 0.941747572815534
KNN -  InfoGainAttributeVal PVD : f1-macro 0.7508064516129032
KNN -  InfoGainAttributeVal PVD : f1-micro 0.912621359223301
NB -  InfoGainAttributeVal PVD : f1-macro 0.5595394736842105
NB -  InfoGainAttributeVal PVD : f1-micro 0.8737864077669902
RF -  InfoGainAttributeVal PVD : f1-macro 0.8378443838934302
RF -  InfoGainAttributeVal PVD : f1-micro 0.9368932038834952
SVM -  All Venous Insufficiency : f1-macro 0.47237569060773477
SVM -  All Venous Insufficie

C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


SVM -  SelectKBest Venous Insufficiency : f1-macro 0.6086065573770492
SVM -  SelectKBest Venous Insufficiency : f1-micro 0.93717277486911
KNN -  SelectKBest Venous Insufficiency : f1-macro 0.48097826086956524
KNN -  SelectKBest Venous Insufficiency : f1-micro 0.9267015706806283
NB -  SelectKBest Venous Insufficiency : f1-macro 0.48097826086956524
NB -  SelectKBest Venous Insufficiency : f1-micro 0.9267015706806283
RF -  SelectKBest Venous Insufficiency : f1-macro 0.48097826086956524
RF -  SelectKBest Venous Insufficiency : f1-micro 0.9267015706806283
SVM -  InfoGainAttributeVal Venous Insufficiency : f1-macro 0.6877384196185286
SVM -  InfoGainAttributeVal Venous Insufficiency : f1-micro 0.9528795811518325
KNN -  InfoGainAttributeVal Venous Insufficiency : f1-macro 0.6738387978142076
KNN -  InfoGainAttributeVal Venous Insufficiency : f1-micro 0.9476439790575917
NB -  InfoGainAttributeVal Venous Insufficiency : f1-macro 0.4837837837837838
NB -  InfoGainAttributeVal Venous Insufficiency :

C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  0 137] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\mclop\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


SVM -  SelectKBest GERD : f1-macro 0.8155901116427433
SVM -  SelectKBest GERD : f1-micro 0.8918918918918919
KNN -  SelectKBest GERD : f1-macro 0.43768996960486317
KNN -  SelectKBest GERD : f1-micro 0.7783783783783784
NB -  SelectKBest GERD : f1-macro 0.48969221835075494
NB -  SelectKBest GERD : f1-micro 0.7945945945945945
RF -  SelectKBest GERD : f1-macro 0.8299632352941176
RF -  SelectKBest GERD : f1-micro 0.9027027027027028
SVM -  InfoGainAttributeVal GERD : f1-macro 0.7888127853881278
SVM -  InfoGainAttributeVal GERD : f1-micro 0.8594594594594595
KNN -  InfoGainAttributeVal GERD : f1-macro 0.6032475490196079
KNN -  InfoGainAttributeVal GERD : f1-micro 0.772972972972973
NB -  InfoGainAttributeVal GERD : f1-macro 0.460448782810547
NB -  InfoGainAttributeVal GERD : f1-micro 0.7351351351351352
RF -  InfoGainAttributeVal GERD : f1-macro 0.8369086100499559
RF -  InfoGainAttributeVal GERD : f1-micro 0.8864864864864865


# Word Embeddings

In [16]:
Encoder = LabelEncoder()
Tfidf_vect = TfidfVectorizer(max_features=600)
Tfidf_vect_NS = TfidfVectorizer(max_features = 600, stop_words = cachedStopWords)

max_tokens = 600

for _,disease in enumerate(disease_list):
    for _, embedding in enumerate(embedding_list):
        disease_data_df = all_df_expanded [all_df_expanded ['disease'] == disease]

        

        if embedding == 'GloVe':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['vector_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_GloVe(X_train)
            X_test = vectorize_batch_GloVe(X_test)
        if embedding == 'FastText':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['vector_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_FastText(X_train)
            X_test = vectorize_batch_FastText(X_test)
        if embedding == 'USE':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['sentence_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_USE(X_train)
            X_test = vectorize_batch_USE(X_test)
            
            vectorize_batch_USE

        Train_Y  = Encoder.fit_transform(y_train)
        Test_Y  = Encoder.fit_transform(y_test)

        performSVM(X_train, X_test, Train_Y, Test_Y, embedding)

        performKNN(X_train, X_test, Train_Y, Test_Y, embedding)

        #performNB(X_train, X_test, Train_Y, Test_Y, embedding)

        performRF(X_train, X_test, Train_Y, Test_Y, embedding)

SVM -  GloVe Asthma : f1-macro 0.47019867549668876
SVM -  GloVe Asthma : f1-micro 0.8875
KNN -  GloVe Asthma : f1-macro 0.47019867549668876
KNN -  GloVe Asthma : f1-micro 0.8875
RF -  GloVe Asthma : f1-macro 0.47019867549668876
RF -  GloVe Asthma : f1-micro 0.8875
SVM -  FastText Asthma : f1-macro 0.4618834080717488
SVM -  FastText Asthma : f1-micro 0.8583333333333333
KNN -  FastText Asthma : f1-macro 0.4618834080717488
KNN -  FastText Asthma : f1-micro 0.8583333333333333
RF -  FastText Asthma : f1-macro 0.4618834080717488
RF -  FastText Asthma : f1-micro 0.8583333333333333


NameError: name 'x' is not defined